<a href="https://colab.research.google.com/github/franklinscudder/DiffieHellmanExample/blob/main/DiffieHellman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install numba --upgrade
!pip3 install pycryptodome

     |████████████████████████████████| 3.4 MB 7.9 MB/s 
     |████████████████████████████████| 25.3 MB 74.0 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2
     |████████████████████████████████| 1.9 MB 8.7 MB/s 


In [8]:
"""
A demo of Diffie-Hellman key exchange using prime modulo exponentiation and 
elliptic curve cryptography.

by Tom Findlay
"""

import random
from math import gcd as gcd
import Crypto.PublicKey.ECC as ecc
from sympy import randprime

class user:
    """
    A class representing a user on one end of the key exchange process
    """
    def __init__(self):
        self.rx = []  # the receiving buffer
        self.secret = random.randint(2 ** 8, 2 ** 16) # a secret integer
    
    def initDH(self, target, bits):
        """
        Start a prime modulus exponentiation key exchange with user instance
        'target' and a prime of size 'bits'.
        """
        print(f"Finding a {bits}-bit prime...")
        self.p = randprime(2 ** (bits-4), 2 ** bits)
        print(f"Finding a suitable generator...")
        self.g = random.randint(2 ** (bits-4), 2 ** bits)
        print(f"The shared prime is: {self.p}")
        print(f"The shared generator is: {self.g}")        
        self.myPubKey = pow(self.g, self.secret, self.p)
        print(f"The initialiser's public key is: {self.myPubKey}")
        target.rx.append(self.p)
        target.rx.append(self.g)
        target.rx.append(self.myPubKey)
        target.respDH(self)
        self.theirPubKey = self.rx.pop()
        print(f"The respondant's public key is: {self.theirPubKey}")
        self.finalKey = pow(self.theirPubKey, self.secret, self.p)
        print(f"The final agreed private key is: {self.finalKey}")

    def respDH(self, target):
        """
        Respond to a prime modulo exponentiation key exchange request.
        (This method is called automatically by the initialising instance.)
        """
        self.theirPubKey = self.rx.pop()
        self.g = self.rx.pop()
        self.p = self.rx.pop()
        self.myPubKey = pow(self.g, self.secret, self.p)
        target.rx.append(self.myPubKey)
        self.finalKey = pow(self.theirPubKey, self.secret, self.p)
        print(f"Respondant's final agreed private key is: {self.finalKey}")

    def initECDHE(self, target):
        """
        Start a elliptic curve key exchange with user instance
        'target'.
        """
        print("Using NIST P-256 elliptic curve...")
        print("Generating a new initialiser key pair...")
        privEC = ecc.generate(curve="p256")
        pubEC = privEC.public_key()
        print("The initialiser's private key is:", privEC)
        print("Sending initialiser's public key...")
        target.rx.append(pubEC)
        target.respECDHE(self)
        theirPubEC = self.rx.pop()
        print("The respondant's public key is:", theirPubEC)
        finalEC = (privEC.d * theirPubEC.pointQ)
        finalECx = finalEC.x
        print(f"Final shared secret is: {finalECx}")
        
    def respECDHE(self, target):
        """
        Respond to an elliptic curve key exchange request.
        (This method is called automatically by the initialising instance.)
        """
        print("Generating a new respondant key pair...")
        privEC = ecc.generate(curve="p256")
        print("The Respondant's private key is:", privEC)
        pubEC = privEC.public_key()
        theirPubEC = self.rx.pop()
        print("The initialiser's public key is: ", theirPubEC)
        print("Sending respondant's public key...")
        target.rx.append(pubEC)
        finalEC = privEC.d * theirPubEC.pointQ
        finalECx = finalEC.x
        print(f"Respondant's final shared secret is: {finalECx}")
        

if __name__ == "__main__":
    alice = user()
    bob = user()

    print("+++ Modulo Exponential Diffie-Hellman +++")
    alice.initDH(bob, 256)
    print()

    print("+++ Elliptic Curve Diffie-Hellman (Ephemeral) +++")
    bob.initECDHE(alice)


+++ Modulo Exponential Diffie-Hellman +++
Finding a 256-bit prime...
Finding a suitable generator...
The shared prime is: 37551529316440649303261682923895156437984339346473081181739941568640677980053
The shared generator is: 7247522214810185171880671422083067885359667150058117088855809525271639043250
The initialiser's public key is: 23913160594869583164062417320976856786957381604358329080985835340754598444219
Respondant's final agreed private key is: 17446349235076518458581185693364516384676857461535740685528333487101862126567
The respondant's public key is: 10618800241902745616277074208898481753970036420891385451762465614196594461546
The final agreed private key is: 17446349235076518458581185693364516384676857461535740685528333487101862126567

+++ Elliptic Curve Diffie-Hellman (Ephemeral) +++
Using NIST P-256 elliptic curve...
Generating a new initialiser key pair...
The initialiser's private key is: EccKey(curve='NIST P-256', point_x=47583724349536376443877466740654162148956889209512